In [1]:
pip install pandas nltk scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import os
import pandas as pd

# Specify the paths to 'spam' and 'ham' folders
spam_path = r"C:\Users\tipur\Downloads\Compressed\enron1\spam"
ham_path = r"C:\Users\tipur\Downloads\Compressed\enron1\spam"

# Function to read emails from a folder and label them
def read_emails(folder, label):
    emails = []
    for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), 'r', encoding='latin1') as file:
            content = file.read()
            emails.append((content, label))
    return emails

# Read spam and ham emails
spam_emails = read_emails(spam_path, 'spam')
ham_emails = read_emails(ham_path, 'ham')

# Create a DataFrame
df = pd.DataFrame(spam_emails + ham_emails, columns=['content', 'label'])


In [5]:
df

,content,label
0,Subject: dobmeos with hgh my energy level has ...,spam
1,Subject: your prescription is ready . . oxwq s...,spam
2,Subject: get that new car 8434\npeople nowthe ...,spam
3,"Subject: await your response\ndear partner ,\n...",spam
4,"Subject: coca cola , mbna america , nascar par...",spam
...,...,...
2995,Subject: our pro - forma invoice attached\ndiv...,ham
2996,Subject: str _ rndlen ( 2 - 4 ) } { extra _ ti...,ham
2997,Subject: check me out !\n61 bb\nhey derm\nbbbb...,ham
2998,Subject: hot jobs\nglobal marketing specialtie...,ham


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

# Function to preprocess a single email
def preprocess_email(email):
    # Convert to lowercase
    email = email.lower()
    # Tokenization
    tokens = word_tokenize(email)
    # Remove stop words and perform stemming
    tokens = [porter.stem(token) for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to all emails
df['processed_content'] = df['content'].apply(preprocess_email)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['processed_content'], df['label'], test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tipur\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tipur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_vectorized = vectorizer.transform(X_test)


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [10]:
# Initialize the Naive Bayes model
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train_vectorized, y_train)


MultinomialNB()

In [11]:
# Make predictions on the test set
y_pred = nb_model.predict(X_test_vectorized)


In [12]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('\nClassification Report:\n', report)


Accuracy: 0.16

Classification Report:
               precision    recall  f1-score   support

         ham       0.10      0.10      0.10       287
        spam       0.21      0.22      0.22       313

    accuracy                           0.16       600
   macro avg       0.16      0.16      0.16       600
weighted avg       0.16      0.16      0.16       600



In [13]:
import joblib

# Save the trained model
joblib.dump(nb_model, 'spam_classifier_model.pkl')

# Save the vectorizer for later use in the web app
joblib.dump(vectorizer, 'email_vectorizer.pkl')


['email_vectorizer.pkl']